In [1]:
# ================================
# Instalação e importações
# ================================
!pip install fastf1

import os
import pandas as pd
import fastf1
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np

# Criar cache
os.makedirs("/content/f1_cache", exist_ok=True)
fastf1.Cache.enable_cache('/content/f1_cache')

# ================================
# Funções auxiliares
# ================================
def to_seconds(td):
    if pd.isnull(td):
        return None
    return td.total_seconds()

def create_dataset(laps, le=None, fit_le=False):
    df = pd.DataFrame({
        'Driver': laps['Driver'],
        'LapNumber': laps['LapNumber'],
        'LapTime': laps['LapTime'].apply(to_seconds),
        'Sector1': laps['Sector1Time'].apply(to_seconds),
        'Sector2': laps['Sector2Time'].apply(to_seconds),
        'Sector3': laps['Sector3Time'].apply(to_seconds),
        'Compound': laps['Compound'].str.upper()
    }).dropna()

    if fit_le:
        df['Compound'] = df['Compound'].fillna('HARD')
        df['CompoundCode'] = le.fit_transform(df['Compound'])
        return df
    else:
        # Substitui compostos desconhecidos pelo composto mais frequente
        df['Compound'] = df['Compound'].apply(lambda x: x if x in le.classes_ else 'HARD')
        df['CompoundCode'] = le.transform(df['Compound'])
        return df

# ================================
# CONFIGURAÇÃO DE CORRIDAS E ANOS
# ================================
GP = 'Bahrain'
ANO_TREINO = 2023
ANO_TESTE = 2024

# ================================
# CARREGAR DADOS DE TREINO
# ================================
train_session = fastf1.get_session(ANO_TREINO, GP, 'R')
train_session.load()
laps_train = train_session.laps

# CARREGAR DADOS DE TESTE
test_session = fastf1.get_session(ANO_TESTE, GP, 'R')
test_session.load()
laps_test = test_session.laps

# ================================
# LabelEncoder universal
# ================================
todos_compostos = pd.concat([laps_train['Compound'], laps_test['Compound']]).str.upper().unique()
le = LabelEncoder()
le.fit(todos_compostos)

df_train = create_dataset(laps_train, le=le, fit_le=True)
df_train['GoodLap'] = df_train.groupby('Driver')['LapTime'].transform(lambda x: (x < x.mean()).astype(int))
feature_cols = ['LapNumber', 'Sector1', 'Sector2', 'Sector3', 'CompoundCode']
X_train = df_train[feature_cols]
y_train = df_train['GoodLap']

df_test = create_dataset(laps_test, le=le, fit_le=False)
df_test['GoodLap'] = df_test.groupby('Driver')['LapTime'].transform(lambda x: (x < x.mean()).astype(int))
X_test = df_test[feature_cols]
y_test = df_test['GoodLap']

# ================================
# TREINAR MODELO
# ================================
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

print("📊 Acurácia na corrida de teste:", accuracy_score(y_test, rf.predict(X_test)))

# ================================
# SIMULAÇÃO DE PITSTOPS PROBABILÍSTICA
# ================================
total_voltas = int(df_test['LapNumber'].max())
durabilidade = {"SOFT": 15, "MEDIUM": 25, "HARD": 35}  # voltas médias por composto
recomendacoes = []

for driver in df_test['Driver'].unique():
    driver_laps = df_test[df_test['Driver'] == driver].sort_values('LapNumber').reset_index(drop=True)

    for idx, row in driver_laps.iterrows():
        lap = int(row['LapNumber'])
        ultima_volta = row

        melhores = []
        for comp in le.classes_:
            X_pneu = pd.DataFrame({
                'LapNumber': [lap],
                'Sector1': [ultima_volta['Sector1']],
                'Sector2': [ultima_volta['Sector2']],
                'Sector3': [ultima_volta['Sector3']],
                'CompoundCode': [le.transform([comp])[0]]
            })
            prob = rf.predict_proba(X_pneu)[0][1]
            pred_novo = np.random.rand() < prob
            melhores.append((comp, pred_novo))

        best_comp, best_pred = max(melhores, key=lambda x: x[1])
        estimativa_pits = total_voltas // durabilidade.get(best_comp, 20)

        recomendacoes.append((
            driver,
            lap,
            best_comp,
            "Boa" if best_pred else "Ruim",
            estimativa_pits
        ))

# ================================
# EXIBIÇÃO COMPLETA
# ================================
df_recomendacoes = pd.DataFrame(
    recomendacoes,
    columns=["Piloto", "Volta Recomendada", "Pneu Sugerido", "Expectativa", "Pitstops Estimados"]
)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

print("\n🔧 Estratégia dinâmica de pitstops probabilística:")
print(df_recomendacoes)

# Exportar CSV
df_recomendacoes.to_csv("estrategia_pitstops.csv", index=False)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.13.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 13.1 which is incompatible.
dataproc-spark-connect 0.8.3 requires websockets>=14.0, but you have websockets 13.1 which is incompatible.


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
INFO:fastf1.fastf1.core:Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	No cached data found for session_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
INFO:fastf1.api:Fetching session info data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_status_d

📊 Acurácia na corrida de teste: 0.8030713640469738

🔧 Estratégia dinâmica de pitstops probabilística:
     Piloto  Volta Recomendada Pneu Sugerido Expectativa  Pitstops Estimados
0       VER                  2          HARD         Boa                   1
1       VER                  3          HARD         Boa                   1
2       VER                  4          HARD         Boa                   1
3       VER                  5        MEDIUM         Boa                   2
4       VER                  6          HARD         Boa                   1
5       VER                  7          HARD         Boa                   1
6       VER                  8          HARD         Boa                   1
7       VER                  9          HARD         Boa                   1
8       VER                 10          HARD         Boa                   1
9       VER                 11          HARD         Boa                   1
10      VER                 12          HARD       